## 인코딩 방법 정리
### 원핫인코딩 (One-Hot Encoding)
- 원핫인코딩은 범주형 데이터를 수치형 데이터로 변환하는 방법입니다. 각 범주를 새로운 열로 나타내며, 해당 범주에 속하면 1, 속하지 않으면 0의 값을 가집니다.
```python
encoded_df = pd.get_dummies(df, columns=['CategoricalColumn'])
```
### 레이블인코딩 (Label Encoding)
레이블인코딩은 범주형 데이터를 순차적인 숫자로 변환하는 방법입니다. 주의해야 할 점은 숫자의 크기에 의미가 없는 경우에는 사용하지 않는 것이 좋습니다.
```python
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['CategoricalColumn'] = label_encoder.fit_transform(df['CategoricalColumn'])
```
### 순서인코딩 (Ordinal Encoding)
순서인코딩은 범주형 데이터를 순서를 고려하여 숫자로 변환하는 방법입니다. 범주 간의 순서가 중요한 경우에 사용됩니다.
```python
order_mapping = {'Low': 0, 'Medium': 1, 'High': 2}
df['CategoricalColumn'] = df['CategoricalColumn'].map(order_mapping)
```
### 타겟인코딩 (Target Encoding)
타겟인코딩은 범주형 데이터를 해당 범주에 대한 타겟 변수의 평균 또는 다른 통계량으로 변환하는 방법입니다. 주로 회귀 문제에서 사용되며, 특정 범주의 타겟 변수 값을 대체하여 모델에 입력합니다.
```python
target_mapping = df.groupby('CategoricalColumn')['TargetColumn'].mean().to_dict()
df['CategoricalColumn'] = df['CategoricalColumn'].map(target_mapping)
```

## 데이터 클리닝
### 1. 결측치 처리
- `df.isnull().sum()` # 결측치 개수 확인
- `df.dropna()` # 결측치가 있는 행 제거
- `df.fillna(0)` # 결측치를 0으로 대체
- `df.fillna(df['Age'].mean(), inplace=True)` # 결측치를 평균값으로 대체
- `df.fillna(df['Age'].median(), inplace=True)` # 결측치를 중앙값으로 대체
- `df.fillna(df['Age'].mode()[0], inplace=True)` # 결측치를 최빈값으로 대체
- `df['Age'].fillna(method='ffill', inplace=True)` # 결측치를 앞의 값으로 대체
- `df['Age'].fillna(method='bfill', inplace=True)` # 결측치를 뒤의 값으로 대체
- `df.interpolate(method='linear', inplace=True)` # 결측치를 선형으로 대체, 시계열 데이터에 적합
- `df.interpolate(method='polynomial', order=2, inplace=True)` # 결측치를 2차 함수로 대체, 시계열 데이터에 적합
- `df.interpolate(method='spline', order=2, inplace=True)` # 결측치를 2차 스플라인으로 대체, 시계열 데이터에 적합
- `df.interpolate(method='krogh', inplace=True)` # 결측치를 Krogh 방법으로 대체, 시계열 데이터에 적합(Krogh는 곡선을 생성하지만 고차원 다항식을 생성하지 않아 과적합을 방지할 수 있음)
- `df.interpolate(method='piecewise_polynomial', order=2, inplace=True)` # 결측치를 2차 다항식으로 대체, 시계열 데이터에 적합
### 다중 대치법
```python
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer = IterativeImputer(max_iter=10, random_state=0, initial_strategy='median', tol=0.001, n_nearest_features=4)
# initial_strategy: 결측치를 대체할 초기값을 설정합니다. 'mean', 'median', 'most_frequent' 중 하나를 선택할 수 있습니다.
# tol: 결측치 대체가 수렴하는 기준을 설정합니다.
# n_nearest_features: 결측치 대체에 사용할 특성의 개수를 설정합니다. (default=None: 모든 특성 사용)

# SalePrice의 중앙값을 이용하여 LotFrontage의 결측치를 다중대체법으로 대체, 컬럼 추가하려면 뒤에 쭉 쓰면 됨
train['LotFrontage'] = imputer.fit_transform(train[['LotFrontage', 'SalePrice']])[:,0]

# 모든 변수를 이용해 결측치를 다중대체법으로 대체 (이 때, 범주형 변수의 경우 인코딩을 먼저 수행해야 함)
train = pd.DataFrame(imputer.fit_transform(train), columns=train.columns)
```

### 2. 이상치 처리
#### IQR(Interquartile Range, 사분범위) 기반 이상치 처리
- `df.describe()` # 기술통계량 확인
- `sns.boxplot(x=df['Column'], y=df['TargetColumn'], fliersize=2.5, whis=1.5)` # 상자그림으로 이상치 확인
```python
#fliersize: 이상치 마커의 크기, whis: 이상치 경계값(기본값 1.5;1.5*IQR)
```
- `Q1 = df['Column'].quantile(0.25)` # 1사분위수
- `Q3 = df['Column'].quantile(0.75)` # 3사분위수
- `IQR = Q3 - Q1` # IQR(Interquartile Range, 사분범위)
- `lower_bound = Q1 - 1.5 * IQR` # 하한값
- `upper_bound = Q3 + 1.5 * IQR` # 상한값
- `df = df[(df['Column'] >= lower_bound) & (df['Column'] <= upper_bound)]` # 이상치 제거
#### Z-Score 기반 이상치 처리
- `from scipy import stats`
- `z_scores = np.abs(stats.zscore(df['Column']))` # Z-Score 계산
- `threshold = 3` # 임계값 설정
- `df = df[(z_scores < threshold)]` # 이상치 제거
#### DBSCAN 기반 이상치 처리
- `from sklearn.cluster import DBSCAN`
- `outlier_detection = DBSCAN(min_samples=2, eps=3)` # eps: 이웃을 정의하는 거리, min_samples: 핵심 샘플의 최소 개수
- `clusters = outlier_detection.fit_predict(df)` # 이상치를 제외한 모든 샘플의 클러스터 레이블을 반환
- `df = df[clusters != -1]` # 이상치 제거
#### Isolation Forest 기반 이상치 처리
- `from sklearn.ensemble import IsolationForest`
- `outlier_detection = IsolationForest(contamination=0.01)` # contamination: 이상치 비율
- `clusters = outlier_detection.fit_predict(df)` # 이상치를 제외한 모든 샘플의 클러스터 레이블을 반환
- `df = df[clusters != -1]` # 이상치 제거
#### Local Outlier Factor 기반 이상치 처리
- `from sklearn.neighbors import LocalOutlierFactor`
- `outlier_detection = LocalOutlierFactor(n_neighbors=2, contamination=0.01)` # n_neighbors: 이웃의 개수, contamination: 이상치 비율
- `clusters = outlier_detection.fit_predict(df)` # 이상치를 제외한 모든 샘플의 클러스터 레이블을 반환
- `df = df[clusters != -1]` # 이상치 제거
```python
# 이상치 제거 예시
from sklearn.ensemble import IsolationForest
outlier_detection = IsolationForest(max_samples = 100, contamination = 0.02, random_state = 12, n_neighbors = 10)
clusters = outlier_detection.fit_predict(train[['LotFrontage', 'LotArea']]) # 연속형 변수만 사용, 결과는 numpy.ndarray, -1은 이상치 1은 정상치
clusters=pd.Series(clusters)
train=pd.concat([train, clusters], axis=1).rename(columns={0:'outlier'})
train = train[train['outlier']!=-1]
```

### 3. 중복 데이터 처리
- `df.duplicated().sum()` # 중복 데이터 개수 확인
- `df.drop_duplicates(inplace=True)` # 중복 데이터 제거

### 4. 특성 공학
- `df['NewColumn'] = df['Column1'] +-*/ df['Column2']` # 새로운 특성 생성
- `df['NewColumn'] = df['Column1'] ** df['Column2']` # 새로운 특성 생성
- `df['NewColumn'] = df['Column1'] % df['Column2']` # 새로운 특성 생성
- `df['NewColumn'] = df['Column1'].apply(lambda x: x**2)` # 새로운 특성 생성
- `df['NewColumn'] = df['Column1'].apply(lambda x: x**0.5)` # 새로운 특성 생성
- `df['NewColumn'] = df['Column1'].apply(lambda x: np.log(x))` # 새로운 특성 생성
- `df['NewColumn'] = df['Column1'].apply(lambda x: np.exp(x))` # 새로운 특성 생성

### 5. 특성 선택
#### 상관관계 기반 특성 선택
- `df.corr()` # 상관관계 확인

In [2]:
import pandas as pd

In [8]:
train = pd.read_csv('train.csv')
train.dropna(axis=0, subset=['LotFrontage'], inplace=True)
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [9]:
from sklearn.ensemble import IsolationForest

outlier_detection = IsolationForest(max_samples = 100, contamination = 0.02, random_state = 12)
clusters = outlier_detection.fit_predict(train[['LotFrontage', 'LotArea']])
clusters=pd.Series(clusters)
train=pd.concat([train, clusters], axis=1).rename(columns={0:'outlier'})
train.head()

c:\Users\qkreo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,outlier
0,1.0,60.0,RL,65.0,8450.0,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0.0,2.0,2008.0,WD,Normal,208500.0,1.0
1,2.0,20.0,RL,80.0,9600.0,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0.0,5.0,2007.0,WD,Normal,181500.0,1.0
2,3.0,60.0,RL,68.0,11250.0,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0.0,9.0,2008.0,WD,Normal,223500.0,1.0
3,4.0,70.0,RL,60.0,9550.0,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0.0,2.0,2006.0,WD,Abnorml,140000.0,1.0
4,5.0,60.0,RL,84.0,14260.0,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0.0,12.0,2008.0,WD,Normal,250000.0,1.0


In [10]:
print(train['LotFrontage'].isnull().sum())
train['LotFrontage'].describe()

210


count    1201.000000
mean       70.049958
std        24.284752
min        21.000000
25%        59.000000
50%        69.000000
75%        80.000000
max       313.000000
Name: LotFrontage, dtype: float64

In [11]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer = IterativeImputer(max_iter=10, random_state=0, initial_strategy='median', tol=0.001, n_nearest_features=15)
#SalePrice의 중앙값을 이용하여 LotFrontage의 결측치를 다중대체법으로 대체
#train['LotFrontage'] = imputer.fit_transform(train[['LotFrontage', 'SalePrice']])[:,0]
train['LotFrontage'] = imputer.fit_transform(train[['LotFrontage', 'SalePrice', 'MSSubClass']])

In [12]:
print(train['LotFrontage'].isnull().sum())
train['LotFrontage'].describe()

0


count    1411.000000
mean       70.043463
std        22.403447
min        21.000000
25%        60.000000
50%        70.006318
75%        79.500000
max       313.000000
Name: LotFrontage, dtype: float64

### 6. 특성 스케일링
#### 표준화, 정규화(Standardization, Normalization)
```python
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

scaler = StandardScaler()
df['Column'] = scaler.fit_transform(df[['Column']])
```
- `StandardScaler`: 평균이 0과 표준편차가 1이 되도록 변환, 공식: (X - X의 평균) / X의 표준편차
- `MinMaxScaler`: 최대값이 각각 1, 최소값이 0이 되도록 변환, 공식: (X - X의 최소값) / (X의 최대값 - X의 최소값)
- `RobustScaler`: 중앙값(median)이 0, IQR(interquartile range)이 1이 되도록 변환, 공식: (X - Q2) / (Q3 - Q1)